In [1]:
import math

with open('day_16.txt') as f:
    packet = f.read()

In [2]:
packet

'60552F100693298A9EF0039D24B129BA56D67282E600A4B5857002439CE580E5E5AEF67803600D2E294B2FCE8AC489BAEF37FEACB31A678548034EA0086253B183F4F6BDDE864B13CBCFBC4C10066508E3F4B4B9965300470026E92DC2960691F7F3AB32CBE834C01A9B7A933E9D241003A520DF316647002E57C1331DFCE16A249802DA009CAD2117993CD2A253B33C8BA00277180390F60E45D30062354598AA4008641A8710FCC01492FB75004850EE5210ACEF68DE2A327B12500327D848028ED0046661A209986896041802DA0098002131621842300043E3C4168B12BCB6835C00B6033F480C493003C40080029F1400B70039808AC30024C009500208064C601674804E870025003AA400BED8024900066272D7A7F56A8FB0044B272B7C0E6F2392E3460094FAA5002512957B98717004A4779DAECC7E9188AB008B93B7B86CB5E47B2B48D7CAD3328FB76B40465243C8018F49CA561C979C182723D769642200412756271FC80460A00CC0401D8211A2270803D10A1645B947B3004A4BA55801494BC330A5BB6E28CCE60BE6012CB2A4A854A13CD34880572523898C7EDE1A9FA7EED53F1F38CD418080461B00440010A845152360803F0FA38C7798413005E4FB102D004E6492649CC017F004A448A44826AB9BFAB5E0AA8053306B0CE4D324BB2149ADDA2904028600021909E0AC7

In [3]:
def hex_to_bin(hexadecimal):
    end_length = len(hexadecimal) * 4 #4 bits for each hex
    hex_as_bin = bin(int(hexadecimal,16))
    padded_bin = hex_as_bin[2:].zfill(end_length)
    return padded_bin

def bin_to_dec(binary):
    return int(binary,2)

def make_packet(binary):
    packet = {
        'version': bin_to_dec(binary[:3]),
        'type_id': bin_to_dec(binary[3:6]),
        'children': []
    }
    if packet['type_id'] == 4:
        packet['value'], bit_remainder = read_literal(binary[6:])
        return packet, bit_remainder
    return packet_w_sub_packets(binary, packet)

def read_literal(binary): # type id = 4
    bit_remainder = binary
    last_group = False
    literal_bin = ''
    while last_group != True:
        last_group = bit_remainder[0] == '0'
        literal_bin += bit_remainder[1:5]
        bit_remainder = bit_remainder[5:]
    return bin_to_dec(literal_bin), bit_remainder

def get_version(packet):
    return (packet['version'] + sum(get_version(child) for child in packet['children']))

def packet_w_sub_packets(binary, packet):
    length_type_id = binary[6]
    if length_type_id == '0': # length of sub packets in next 15 bits
        length_subpackets = bin_to_dec(binary[7:22])
        sub_remainder = binary[22:(22+length_subpackets)]
        bit_remainder = binary[22+length_subpackets:] #should be trailing 0's?
        while len(sub_remainder) > 0:
            sub_packet, sub_remainder = make_packet(sub_remainder)
            packet['children'].append(sub_packet)
    else: # number of subpackets contained in current packet (11 bits)
        num_subpackets = bin_to_dec(binary[7:18])
        bit_remainder = binary[18:]
        for _ in range(num_subpackets):
            sub_packet, bit_remainder = make_packet(bit_remainder)
            packet['children'].append(sub_packet)
    return packet, bit_remainder

def get_value(packet):
    if packet['type_id'] == 0:
        return sum(get_value(child) for child in packet['children'])
    if packet['type_id'] == 1:
        return math.prod(get_value(child) for child in packet['children'])
    if packet['type_id'] == 2:
        return min(get_value(child) for child in packet['children'])
    if packet['type_id'] == 3:
        return max(get_value(child) for child in packet['children'])
    if packet['type_id'] == 4:
        return packet['value']
    if packet['type_id'] == 5:
        return get_value(packet['children'][0]) > get_value(packet['children'][1])
    if packet['type_id'] == 6:
        return get_value(packet['children'][0]) < get_value(packet['children'][1])
    if packet['type_id'] == 7:
        return get_value(packet['children'][0]) == get_value(packet['children'][1])

In [4]:
tree, _ = make_packet(hex_to_bin(packet))
version_sum = get_version(tree)
root_value = get_value(tree)
print('Part 1:', version_sum)
print('Part 2:', root_value)

Part 1: 877
Part 2: 194435634456
